In [ ]:
#pickles : gini_table, demo2018, margins2018, area table, comp table, rural table


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline


In [1]:
gini_table = pd.read_pickle('gini_table')
demo2018 = pd.read_pickle('demo2018')
margins2018 = pd.read_pickle('margins2018')
area_table = pd.read_pickle('area_table')
comp_table = pd.read_pickle('comp_table')
rural_table = pd.read_pickle('rural_table')
gerry_index = pd.read_pickle('gerryindex')

In [ ]:
gerry_index.sort_values(by=['district']).tail(5)

In [ ]:
#Merging  Tables
df = margins2018.merge(gini_table, left_on='district', right_on='district')
df = df.merge(rural_table, left_on='district', right_on='district')
df = df.drop(['state'], axis=1)


In [ ]:
df = df.merge(area_table, left_on='district', right_on='district')
df = df.merge(demo2018, left_on='district', right_on='Code')
df = df.merge(gerry_index, left_on='district', right_on='district')

In [2]:
dfa = df.set_index('district')

In [3]:
dfa.rename(columns={'Land area (mi²)':'land_area_mi'}, inplace=True)

In [ ]:
# format numbers to numeric
dfa['land_area_mi'] = dfa['land_area_mi'].apply(lambda x: float(x.replace(',','')))
dfa['median_income'] = dfa['median_income'].apply(lambda x: x.replace('$',''))
dfa['median_income'] = dfa['median_income'].apply(lambda x: float(x.replace(',','')))

In [ ]:
dfa[['land_area_mi', 'median_income']] = dfa[['land_area_mi', 'median_income']].apply(pd.to_numeric)

In [ ]:
dfamod = dfa.drop(['2016 Clinton Margin', 'clinton2016', 'trump2016', 'obama2012', 'romney2012', '2016dem', '2016rep', '2014dem', '2014rep','state', 'Code', 'First Elected', 'Birth Year', 'LGBTQ', '2018dem', '2018rep'],axis=1)

In [ ]:
dfamod1 = dfamod.copy()

In [ ]:
# categorical incumbent 
dfamod1['Pre-2018 Incumbent'] = dfamod1['Pre-2018 Incumbent'].apply(lambda x: 1 if x == 'Won' else 0)

# narrowing religions 
dfamod1['Religion'] = dfamod1['Religion'].apply(lambda x: 'catholic' if x == 'Christian - Roman Catholic' or x == 'Christian - Chaldean Catholic' else 'mormon' if x == 'Christian - Mormon' else 'jewish' if x == 'Jewish' else 'other' if x == 'Unknown/Refused' or x == 'Muslim' or x == 'Buddhist - Soka Gakkai' or x == 'Hindu' else 'protestant')

# narrowing race
dfamod1['Race/ Ethnicity'] = dfamod1['Race/ Ethnicity'].apply(lambda x: 'white' if x == 'White - Non-Hispanic' else 'non_white')

In [ ]:
# calc density
dfamod1['density'] = dfamod1['population'] / dfamod1['land_area_mi']

In [ ]:
# populations should be roughly equal
dfamod1 = dfamod1.drop(['population'], axis=1)

In [ ]:
dfadumf.to_csv('dfmain.csv')
df1 = pd.read_csv('dfmain.csv')

In [ ]:
# plt.figure(figsize=(25,15))
# cor_plot = sns.heatmap(df1.corr().round(4), cmap='seismic', annot=True, vmin=-1, vmax=1);

In [ ]:
#sns_plot = sns.pairplot(dfamod)

In [ ]:
#sns_plot.savefig('pair.png')

In [ ]:
#set dummies
dfadum = pd.get_dummies(dfamod1)
dfadum.columns

In [ ]:
dfadumf = dfadum.fillna(dfadum.mean())

In [ ]:
# set features and target for modeling 
features = [c for c in dfadum if c != 'Dem Margin']

X = dfadumf[features]
y = dfadumf['Dem Margin']

In [ ]:
def cv_results(X, y, model=LinearRegression(normalize=True), scoring='r2', rounding=4, cv=5, minimize_score=False):
    '''
    For a set of features and target X, y, perform a 5 fold cross validation.
    Fit and validate a model, and report results
    Note: `sklearn.model_selection.cross_validate` defaults to 5 fold, 80/20 validation splits.
    '''
    
    model_cv = cross_validate(model, X, y, cv=cv, return_train_score=True, 
                              scoring=scoring, return_estimator=True)

    if minimize_score:
        select = model_cv['test_score'].argmin()
    else:
        select = model_cv['test_score'].argmax()

    final_model = model_cv['estimator'][select]
    
    # Capture the ratio between the train and test scores to understand possible under/over fitting
    model_cv['train_test_score_ratio'] = model_cv['train_score'] / model_cv['test_score']
    
    mean_score = round(model_cv['test_score'].mean(), rounding)
    min_score = round(model_cv['test_score'].min(), rounding)
    max_score = round(model_cv['test_score'].max(), rounding)
    
    mean_ratio = round(model_cv['train_test_score_ratio'].mean(), rounding)
    min_ratio = round(model_cv['train_test_score_ratio'].min(), rounding)
    max_ratio = round(model_cv['train_test_score_ratio'].max(), rounding)
    
    # report results
    print(f'Test {scoring} {":":<15} Mean = {mean_score}\tRange = ({min_score}, {max_score})')
    print(f'Train/Test {scoring} Ratio {":":<3} Mean = {mean_ratio}\tRange = ({min_ratio}, {max_ratio})')
    
    print('\nBest Model Feature coefficient results:')
    for feature, coef in zip(X.columns, final_model.coef_):
        print(f'{feature + ":":<16} {coef:.2f}') 

In [ ]:
cv_results(X, y)

In [ ]:
X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=10)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.25, random_state=3)

In [ ]:
lm = LinearRegression()

#Feature scaling for train, val, and test so that we can run our ridge model on each
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_val_scaled = scaler.transform(X_val.values)
X_test_scaled = scaler.transform(X_test.values)

lm_reg = Ridge(alpha=1)

#Feature transforms for train, val, and test so that we can run our poly model on each
poly = PolynomialFeatures(degree=2) 

X_train_poly = poly.fit_transform(X_train.values)
X_val_poly = poly.transform(X_val.values)
X_test_poly = poly.transform(X_test.values)

lm_poly = LinearRegression()

In [ ]:
lm.fit(X_train, y_train)
print(f'Linear Regression val R^2: {lm.score(X_val, y_val):.3f}')

lm_reg.fit(X_train_scaled, y_train)
print(f'Ridge Regression val R^2: {lm_reg.score(X_val_scaled, y_val):.3f}')

lm_poly.fit(X_train_poly, y_train)
print(f'Degree 2 polynomial regression val R^2: {lm_poly.score(X_val_poly, y_val):.3f}')

In [ ]:
# array for kfold
X, y = np.array(X), np.array(y)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state = 71)
cv_lm_r2s, cv_lm_reg_r2s = [], [] #collect the validation results for both models

for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    #simple linear regression
    lm = LinearRegression()
    lm_reg = Ridge(alpha=1)

    lm.fit(X_train, y_train)
    cv_lm_r2s.append(lm.score(X_val, y_val))
    
    #ridge with feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    lm_reg.fit(X_train_scaled, y_train)
    cv_lm_reg_r2s.append(lm_reg.score(X_val_scaled, y_val))

print('Simple regression scores: ', cv_lm_r2s)
print('Ridge scores: ', cv_lm_reg_r2s, '\n')

print(f'Simple mean cv r^2: {np.mean(cv_lm_r2s):.3f} +- {np.std(cv_lm_r2s):.3f}')
print(f'Ridge mean cv r^2: {np.mean(cv_lm_reg_r2s):.3f} +- {np.std(cv_lm_reg_r2s):.3f}')

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

lm_reg = Ridge(alpha=1)
lm_reg.fit(X_scaled,y)
print(f'Ridge Regression test R^2: {lm_reg.score(X_test_scaled, y_test):.3f}')

In [ ]:
### STATS MODELS
X = dfadumf[features]
y = dfadumf['Dem Margin']

model = sm.OLS(y, sm.add_constant(X)) 

fit = model.fit()
fit.summary()

In [ ]:
### ridge lasso

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=43)

In [ ]:
std = StandardScaler()
std.fit(X_train.values)

In [ ]:
X_tr = std.transform(X_train.values)
X_te = std.transform(X_test.values)

In [ ]:
alphavec = 10**np.linspace(-2,2,200)

lasso_model = LassoCV(alphas = alphavec, cv=5)
lasso_model.fit(X_tr, y_train)

In [ ]:
lasso_model.alpha_

In [ ]:
list(zip(X_train.columns, lasso_model.coef_))

In [ ]:
alphavec = 10**np.linspace(-2,2,200)

ridge_model = RidgeCV(alphas = alphavec, cv=5)
ridge_model.fit(X_tr, y_train)

In [ ]:
ridge_model.alpha_

In [ ]:
list(zip(X_train.columns, ridge_model.coef_))

In [ ]:
# eliminating columns

In [ ]:
#### round again
keep = ['gini', 'per_urban', 'blackper', 'bach_higher', '2016presmarg', '2012presmarg',
        '2016marg', '2014marg', 'time_in_office', 'Party_Democratic']#, 'Religion_catholic', 'Religion_protestant']

X = dfadumf[keep]
y = dfadumf['Dem Margin']

In [ ]:
model = sm.OLS(y, sm.add_constant(X)) 

fit = model.fit()
fit.summary()

In [ ]:
cv_results(X, y)

In [ ]:
X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=10)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.25, random_state=3)

In [ ]:
lm = LinearRegression()

#Feature scaling for train, val, and test so that we can run our ridge model on each
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_val_scaled = scaler.transform(X_val.values)
X_test_scaled = scaler.transform(X_test.values)

lm_reg = Ridge(alpha=1)

#Feature transforms for train, val, and test so that we can run our poly model on each
poly = PolynomialFeatures(degree=2) 

X_train_poly = poly.fit_transform(X_train.values)
X_val_poly = poly.transform(X_val.values)
X_test_poly = poly.transform(X_test.values)

lm_poly = LinearRegression()

In [ ]:
lm.fit(X_train, y_train)
print(f'Linear Regression val R^2: {lm.score(X_val, y_val):.3f}')

lm_reg.fit(X_train_scaled, y_train)
print(f'Ridge Regression val R^2: {lm_reg.score(X_val_scaled, y_val):.3f}')

lm_poly.fit(X_train_poly, y_train)
print(f'Degree 2 polynomial regression val R^2: {lm_poly.score(X_val_poly, y_val):.3f}')

In [ ]:
X, y = np.array(X), np.array(y)
kf = KFold(n_splits=5, shuffle=True, random_state = 71)
cv_lm_r2s, cv_lm_reg_r2s = [], [] #collect the validation results for both models

for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    #simple linear regression
    lm = LinearRegression()
    lm_reg = Ridge(alpha=1)

    lm.fit(X_train, y_train)
    cv_lm_r2s.append(lm.score(X_val, y_val))
    
    #ridge with feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    lm_reg.fit(X_train_scaled, y_train)
    cv_lm_reg_r2s.append(lm_reg.score(X_val_scaled, y_val))

print('Simple regression scores: ', cv_lm_r2s)
print('Ridge scores: ', cv_lm_reg_r2s, '\n')

print(f'Simple mean cv r^2: {np.mean(cv_lm_r2s):.3f} +- {np.std(cv_lm_r2s):.3f}')
print(f'Ridge mean cv r^2: {np.mean(cv_lm_reg_r2s):.3f} +- {np.std(cv_lm_reg_r2s):.3f}')

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

lm_reg = Ridge(alpha=1)
lm_reg.fit(X_scaled,y)
print(f'Ridge Regression test R^2: {lm_reg.score(X_test_scaled, y_test):.3f}')

In [ ]:
X = dfadumf[keep]
y = dfadumf['Dem Margin']

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=43)

In [ ]:
std = StandardScaler()
std.fit(X_train.values)

In [ ]:
alphavec = 10**np.linspace(-2,2,200)

lasso_model = LassoCV(alphas = alphavec, cv=5)
lasso_model.fit(X_tr, y_train)

In [ ]:
lasso_model.alpha_

In [ ]:
list(zip(X_train.columns, lasso_model.coef_))

In [ ]:
alphavec = 10**np.linspace(-2,2,200)

ridge_model = RidgeCV(alphas = alphavec, cv=5)
ridge_model.fit(X_tr, y_train)

In [ ]:
ridge_model.alpha_

In [ ]:
list(zip(X_train.columns, ridge_model.coef_))

In [ ]:
X, y = np.array(X), np.array(y)

X, y = np.array(X), np.array(y)
kf = KFold(n_splits=5, shuffle=True, random_state = 71)
cv_lm_r2s, cv_lm_reg_r2s = [], [] #collect the validation results for both models

for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    #simple linear regression
    lm = LinearRegression()
    lm_reg = Ridge(alpha=37.8)

    lm.fit(X_train, y_train)
    cv_lm_r2s.append(lm.score(X_val, y_val))
    
    #ridge with feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    lm_reg.fit(X_train_scaled, y_train)
    cv_lm_reg_r2s.append(lm_reg.score(X_val_scaled, y_val))

print('Simple regression scores: ', cv_lm_r2s)
print('Ridge scores: ', cv_lm_reg_r2s, '\n')

print(f'Simple mean cv r^2: {np.mean(cv_lm_r2s):.3f} +- {np.std(cv_lm_r2s):.3f}')
print(f'Ridge mean cv r^2: {np.mean(cv_lm_reg_r2s):.3f} +- {np.std(cv_lm_reg_r2s):.3f}')

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

lm_reg = Ridge(alpha=37.8)
lm_reg.fit(X_scaled,y)
print(f'Ridge Regression test R^2: {lm_reg.score(X_test_scaled, y_test):.3f}')

In [ ]:
y_pred = lm_reg.predict(X_test_scaled)

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(y_pred, y_test)
plt.title('Y predicted vs Y test')

In [ ]:
#Mean Absolute Error (MAE)
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true)) 

mae(y_test, y_pred)

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
plt.figure(figsize=(10,7))
res = y_test - y_pred
plt.scatter(y_test, res)
plt.xlabel('Actual Margin')
plt.ylabel('residual')

In [ ]:
#QQ plot
plt.figure(figsize=(30,10))
    

plt.subplot(1, 3, 3)

stats.probplot(res, dist="norm", plot=plt)
plt.title("Normal Q-Q plot")

In [ ]:
#dfadumf.to_csv('dfmain.csv')
df1 = pd.read_csv('dfmain.csv')